In [ ]:
from datasets import load_dataset

review_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Cell_Phones_and_Accessories")
#product_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_Cell_Phones_and_Accessories", split="full")

In [137]:
review_dataset["full"][0] 

{'rating': 4.0,
 'title': 'No white background! It’s clear!',
 'text': 'I bought this bc I thought it had the nice white background. Turns out it’s clear & since my phone is blue it doesn’t look anything like this.  If I had known that I would have purchased something else. It works ok.',
 'images': [{'attachment_type': 'IMAGE',
   'large_image_url': 'https://images-na.ssl-images-amazon.com/images/I/B1PrCo7ZjXS._SL1600_.jpg',
   'medium_image_url': 'https://images-na.ssl-images-amazon.com/images/I/B1PrCo7ZjXS._SL800_.jpg',
   'small_image_url': 'https://images-na.ssl-images-amazon.com/images/I/B1PrCo7ZjXS._SL256_.jpg'}],
 'asin': 'B08L6L3X1S',
 'parent_asin': 'B08L6L3X1S',
 'user_id': 'AFKZENTNBQ7A7V7UXW5JJI6UGRYQ',
 'timestamp': 1612044451196,
 'helpful_vote': 0,
 'verified_purchase': True}

In [138]:
data = review_dataset["full"].select(range(100))
print(type(data))
data

<class 'datasets.arrow_dataset.Dataset'>


Dataset({
    features: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase'],
    num_rows: 100
})

In [139]:
import pandas as pd
from datetime import datetime

# Extract and convert relevant fields
extracted_data = []
for item in data:
    extracted_data.append({
        "star_rating": item.get("rating"),
        "product_title": item.get("title"),
        "review_body": item.get("text"),
        "review_date": datetime.utcfromtimestamp(item.get("timestamp") / 1000).strftime('%Y-%m-%d'),
        "verified_purchase": item.get("verified_purchase")
    })

# Create DataFrame
df = pd.DataFrame(extracted_data)

# Preview the DataFrame
print(df.head(5))


   star_rating                        product_title  \
0          4.0     No white background! It’s clear!   
1          5.0  Awesome!  Great price!  Works well!   
2          5.0   Worked but took an hour to install   
3          4.0                               Decent   
4          5.0                             LOVE IT!   

                                         review_body review_date  \
0  I bought this bc I thought it had the nice whi...  2021-01-30   
1  Perfect. How pissed am I that I recently paid ...  2018-08-16   
2  Overall very happy with the end result. If you...  2021-08-17   
3  Lasted about 9 months then the lock button bro...  2020-05-26   
4  LOVE THIS CASE! Works better than my expensive...  2014-08-25   

   verified_purchase  
0               True  
1               True  
2               True  
3               True  
4               True  


In [ ]:
! pip install textblob
from textblob import TextBlob

# Apply TextBlob polarity
df['sentiment_score'] = df['product_title'].apply(lambda x: TextBlob(x).sentiment.polarity)

# Classify sentiment
df['sentiment_label'] = df['sentiment_score'].apply(
    lambda x: 'positive' if x > 0.05 else 'negative' if x < -0.05 else 'neutral'
)

#print(df[['review_body', 'sentiment_score', 'sentiment_label']].head())
df

,star_rating,product_title,review_body,review_date,verified_purchase,sentiment_score,sentiment_label
0,4.0,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,2021-01-30,True,0.062500,positive
1,5.0,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,2018-08-16,True,1.000000,positive
2,5.0,Worked but took an hour to install,Overall very happy with the end result. If you...,2021-08-17,True,0.000000,neutral
3,4.0,Decent,Lasted about 9 months then the lock button bro...,2020-05-26,True,0.166667,positive
4,5.0,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,2014-08-25,True,0.625000,positive
...,...,...,...,...,...,...,...
95,5.0,More durable than expected,Beautiful paint and stays on very well. Seems ...,2021-01-21,True,0.200000,positive
96,2.0,Caution,Was difficult to get on my iPhone 14 plus. On...,2022-12-31,True,0.000000,neutral
97,5.0,Pretty,Better than expected. Very nice looking case....,2022-12-06,True,0.250000,positive
98,2.0,"Well, it’s a case","This is a thin, cheap case. It was inexpensiv...",2022-11-29,True,0.000000,neutral


In [142]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

# Download the VADER lexicon if not already downloaded
import nltk
nltk.download('vader_lexicon')

# Initialize analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\arpan\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [141]:
# Apply sentiment analysis
df['sentiment_score'] = df['review_body'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Classify sentiment
df['sentiment_label'] = df['sentiment_score'].apply(
    lambda x: 'positive' if x > 0.05 else 'negative' if x < -0.05 else 'neutral'
)

# Preview
#print(df[['product_title','review_body', 'sentiment_score', 'sentiment_label']].head())
df

,star_rating,product_title,review_body,review_date,verified_purchase,sentiment_score,sentiment_label
0,4.0,No white background! It’s clear!,I bought this bc I thought it had the nice whi...,2021-01-30,True,0.8442,positive
1,5.0,Awesome! Great price! Works well!,Perfect. How pissed am I that I recently paid ...,2018-08-16,True,-0.8543,negative
2,5.0,Worked but took an hour to install,Overall very happy with the end result. If you...,2021-08-17,True,0.9872,positive
3,4.0,Decent,Lasted about 9 months then the lock button bro...,2020-05-26,True,0.4767,positive
4,5.0,LOVE IT!,LOVE THIS CASE! Works better than my expensive...,2014-08-25,True,0.9046,positive
...,...,...,...,...,...,...,...
95,5.0,More durable than expected,Beautiful paint and stays on very well. Seems ...,2021-01-21,True,0.7425,positive
96,2.0,Caution,Was difficult to get on my iPhone 14 plus. On...,2022-12-31,True,-0.6124,negative
97,5.0,Pretty,Better than expected. Very nice looking case....,2022-12-06,True,0.9134,positive
98,2.0,"Well, it’s a case","This is a thin, cheap case. It was inexpensiv...",2022-11-29,True,0.8387,positive


In [144]:
print(df[df['sentiment_label']=='neutral'][['star_rating','review_body','sentiment_label','product_title']])
#df.iloc[46]

    star_rating                                        review_body  \
6           5.0  Item came as described! Fast shipping & east t...   
23          5.0                                      Fast charging   
46          5.0                                 It works great!!!!   
50          3.0  Doesn't cover the entire screen, does its job ...   
55          1.0  Fell apart right away.  Clamps get in the way ...   
56          5.0  Have bought this brand for a few years. I have...   
61          5.0                      Durable.  Just what I needed.   
62          3.0  Too slippery.  I dropped my phone constantly. ...   

   sentiment_label                                      product_title  
6          neutral                       Great item! Easy to install!  
23         neutral                                               Fast  
46         neutral                                         Five Stars  
50         neutral  Not so crazy cool, but works for now until I d...  
55       